# Kings of Indigo Scraping

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from random import randint
import time
import urllib
import json

C:\Users\linds\anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
# Headers to imitate browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.146 Mobile Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.5',
    'DNT': '1',
    'Referer': 'https://www.kingsofindigo.com/products',
    'Upgrade-Insecure-Requests': '1'
}

In [3]:
main_URL = 'https://kingsofindigo.com'
URL_extension_for_products = '/products.json?limit=250&page='
current_page_number = 1


all_URLs = []
current_product_list = None


while current_product_list != []:
    # make the full json url
    url = main_URL + URL_extension_for_products + str(current_page_number)
    print(url)
    
    # Get the page
    req = urllib.request.Request(url, headers=headers)
    with urllib.request.urlopen(req) as request:
        s = request.read()
        response = json.loads(s)
        current_product_list = response['products']
    
    time.sleep(randint(4,6))
    
    # construct the URL
    for product in current_product_list:
        product_url = main_URL + '/products/' + product['handle']
        all_URLs.append(product_url)
        
    current_page_number += 1

https://kingsofindigo.com/products.json?limit=250&page=1
https://kingsofindigo.com/products.json?limit=250&page=2
https://kingsofindigo.com/products.json?limit=250&page=3
https://kingsofindigo.com/products.json?limit=250&page=4
https://kingsofindigo.com/products.json?limit=250&page=5


In [4]:
print(all_URLs)
print(len(all_URLs))

['https://kingsofindigo.com/products/silvio-clean-xavier-stay-black', 'https://kingsofindigo.com/products/daria-dark-indigo', 'https://kingsofindigo.com/products/kids-koi-jacket-dry-re-gen-selvage', 'https://kingsofindigo.com/products/kids-koi-dry-re-gen-selvage', 'https://kingsofindigo.com/products/adam-triple-r-quilted-blanket', 'https://kingsofindigo.com/products/daniel-dark-indigo-cord', 'https://kingsofindigo.com/products/shotoku-triple-r-quilted-blanket', 'https://kingsofindigo.com/products/fiorella-triple-r-quilted-blanket', 'https://kingsofindigo.com/products/david-triple-r-quilted-blanket', 'https://kingsofindigo.com/products/caroline-dark-indigo-cord', 'https://kingsofindigo.com/products/taja-chambray-tencel', 'https://kingsofindigo.com/products/john-clean-medium-used', 'https://kingsofindigo.com/products/alice-eco-recycled-dark-used', 'https://kingsofindigo.com/products/leila-patchwork-anubha-fabrics', 'https://kingsofindigo.com/products/kong-eco-recycled-blue-used', 'https:

In [5]:
def find_product_name(soup):
    
    product_title = soup.find('h1', class_ = 'ProductMeta__Title').text
    
    return product_title

In [6]:
def find_description(soup):
    description = soup.find('div', class_ = 'ProductMeta__Description')

    description_headers = description.find_all('h3')

    for items in description_headers:
        items.extract()
        
    description_list = [description.text.strip()]
        
    description_list = [item.replace('\r\n', ' ').replace('\r', ' ').replace('\n', ' ') for item in description_list]
    
    return description_list

In [7]:
def find_material_color_and_extradescription(soup):
    '''This function parses that table on the page and extracts the following: 
    a list of materials, the color, and a description list made of the remaining information'''
    
    material_holder = {'Description': []}
    
    
    material_container = soup.find_all('div', class_ = 'Product__material Grid__Cell 1/4--desk')

    collected_rows = []
    individual_rows = []

    for container in material_container:
        collected_rows.append(container.find_all('tr'))


    # Flatten the array of arrays into just an array of rows
    for set_of_rows in collected_rows:
        for row in set_of_rows:
            individual_rows.append(row)


    table_data = []

    # Put each td tag into a list
    for row in individual_rows:
        table_data.append(row.find_all('td'))
        
        
    table_data_cleaned = {}

    # Strip and remove the colon:
    for item in table_data:
        first = item[0].text.strip()
        second = item[1].text.strip()

        if first[- 1] == ":" :
            first = first[0: - 1]

        table_data_cleaned[first] = second
        
    # Don't need the Article Number, get rid of it
    table_data_cleaned.pop('Articlenumber')
        
    
    for field in table_data_cleaned:
        if field == 'Quality' or field == 'Color':
            material_holder[field] = table_data_cleaned[field]
        else:
            description_item = field + ": " + table_data_cleaned[field]
            material_holder['Description'].append(description_item)


    the_materials_as_string = None
    the_materials_split = []
    material_list_cleaned = []

    if 'Quality' in table_data_cleaned:
        the_materials_as_string = table_data_cleaned['Quality']

        if "+" in the_materials_as_string:
            current_materials = the_materials_as_string.split('+')
            for material in current_materials:
                material_list_cleaned.append(material.strip())

        elif ", " in the_materials_as_string:
            current_materials = the_materials_as_string.split(', ')
            for material in current_materials:
                material_list_cleaned.append(material.strip())

        else:
            the_materials_split = the_materials_as_string.split(' ')
            current_material = ""
            how_many_digits = 0

            for item in the_materials_split:

                if item[0].isdigit() and how_many_digits == 0:
                    current_material = current_material + ' ' + item
                    how_many_digits += 1

                elif item[0].isdigit() and how_many_digits == 1:
                    material_list_cleaned.append(current_material.strip())
                    current_material = ""
                    current_material = current_material + ' ' + item

                else:
                    current_material = current_material + ' ' + item
                    if item == the_materials_split[-1]:
                        material_list_cleaned.append(current_material.strip())
                        
        material_list_cleaned = [item.replace(",", ".") for item in material_list_cleaned]
        
        material_holder['Material'] = material_list_cleaned

    material_holder.pop('Quality')

    
    return material_holder

In [8]:
def find_price(soup):

    price_container = soup.find('div', class_ = 'ProductMeta__PriceList Heading')
    prices = price_container.find_all('span', class_ = 'Price')
    
    price_list = []
    for item in prices:
        price_list.append(item.text)
    
    return prices[-1].text.replace(",", ".")

In [9]:
def find_image(soup):
    image_url_container = soup.find('div', class_ = 'custom-image-carousel__imageContainer').find('img')

    image_url_attributes = image_url_container.attrs

    image_url = 'https:' + image_url_attributes['data-original-src']

    return image_url

In [10]:
def scrape_single_product(URL, html_soup):
    product = {}
    # Name, Material, Color, Price, URL, Image, Brand_name, Description
    # string, list, string, float, string, string, string, list
    
    product['Name'] = find_product_name(html_soup)
    
    # Do Description here so it can be before the extra description items. 
    # Will need to remove the " material_holder = {'Description': []}" from the find_material_color_and_description fn.
    
    product['Description'] = find_description(html_soup)
    
    #Materials, Color, and Extras for Description.
    material_color_and_extradescriptions = find_material_color_and_extradescription(html_soup)
    
    product['Material'] = material_color_and_extradescriptions['Material']
    
    product['Color'] = material_color_and_extradescriptions['Color']
    
    for item in material_color_and_extradescriptions['Description']:
        product["Description"].append(item)
    
    
    product['Price'] = find_price(html_soup)
    
    product['URL'] = URL

    product['Image'] = find_image(html_soup)
    
    product['Brand_name'] = 'Kings Of Indigo'

    return product


In [11]:
all_products = []
failed_urls = []

for url in all_URLs:
    try:

        req = urllib.request.Request(url, headers=headers)
        
        with urllib.request.urlopen(req) as request:
            page = request.read()
            soup = bs(page, 'html.parser')
            all_products.append(scrape_single_product(url, soup))
            
            time.sleep(randint(5,15))
            
    except:
        failed_urls.append(url)

print(failed_urls)
print(len(all_products))
print('Done!')

In [ ]:
fail_again_URLs = []
for url in failed_urls:
    try:

        req = urllib.request.Request(url, headers=headers)
        
        with urllib.request.urlopen(req) as request:
            page = request.read()
            soup = bs(page, 'html.parser')
            all_products.append(scrape_single_product(url, soup))
            
            time.sleep(randint(5,15))
            
    except:
        fail_again_URLs.append(url)

In [ ]:
print(len(all_products))

903


In [ ]:
# product_data_frame = pd.DataFrame(data = product_as_list, columns = ['Name', 'Material', 'Color', 'Price', 'URL', 'Image', 'Brand Name', 'Description'])
product_data_frame = pd.DataFrame(data = all_products)

product_data_frame.to_csv('KingsOfIndigo_table.csv', encoding='utf-8-sig')

pd.DataFrame(data = all_products)

,Name,Description,Material,Color,Price,URL,Image,Brand_name
0,BARD,[Worker shirt with a regular fit and tradition...,[100% Tencel lyocell],BLUE BLACK TENCEL EMBROIDERY,€299.99,https://kingsofindigo.com/products/bard-blue-b...,https://cdn.shopify.com/s/files/1/0060/4553/22...,Kings Of Indigo
1,ATHALIE,[Loose fitted blouse with V-neck and double la...,[100% Tencel lyocell],BLUE BLACK TENCEL EMBROIDERY,€199.99,https://kingsofindigo.com/products/athalie-blu...,https://cdn.shopify.com/s/files/1/0060/4553/22...,Kings Of Indigo
2,LUCIUS SHORT,[Lucius is a five pocket denim short. Vintage ...,[100% Organic cotton],LIGHT MARBLE,€99.95,https://kingsofindigo.com/products/lucius-shor...,https://cdn.shopify.com/s/files/1/0060/4553/22...,Kings Of Indigo
3,KIDS KOI JACKET,[Kings Of Indigo presents an exclusive capsule...,"[96% Organic cotton, 4% Natural rubber]",BIO STRETCH BLUE BADGE ALLOVER,€199.99,https://kingsofindigo.com/products/kids-koi-ja...,https://cdn.shopify.com/s/files/1/0060/4553/22...,Kings Of Indigo
4,BABY KOI,[Kings Of Indigo presents an exclusive capsule...,"[96% Organic cotton, 4% Natural rubber]",BIO STRETCH BLUE BADGE ALLOVER,€149.99,https://kingsofindigo.com/products/baby-koi-bi...,https://cdn.shopify.com/s/files/1/0060/4553/22...,Kings Of Indigo
...,...,...,...,...,...,...,...,...
898,JUNO,[A slim fit with a mid rise. Juno has been wit...,"[83% Organic cotton, 15% recycled cotton, 2% e...",BLACK WORN IN,€129.95,https://kingsofindigo.com/products/juno-black-...,https://cdn.shopify.com/s/files/1/0060/4553/22...,Kings Of Indigo
899,DANIEL,[Relaxed tapered - Daniel is a relaxed tapered...,"[81% Organic cotton, 15% recycled cotton, 3% p...",STAY BLACK,€109.95,https://kingsofindigo.com/products/daniel-stay...,https://cdn.shopify.com/s/files/1/0060/4553/22...,Kings Of Indigo
900,JUNO,"[A slim fit with a mid rise, Juno has been wit...","[98% Organic cotton, 2% elastane]",MID INDIGO,€129.95,https://kingsofindigo.com/products/juno-mid-in...,https://cdn.shopify.com/s/files/1/0060/4553/22...,Kings Of Indigo
901,CHRISTINA HIGH,[Christina High is super skinny. Christina is ...,"[80% Organic cotton, 13% elastomultiester, 7% ...",TWO WAY BLUE,€149.95,https://kingsofindigo.com/products/christina-h...,https://cdn.shopify.com/s/files/1/0060/4553/22...,Kings Of Indigo
